In [1]:
example = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
""".strip().splitlines()

with open("day07.txt", "r") as f:
    data = f.readlines()

In [2]:
from typing import Dict
from re import match, findall

class Bag(object):
    def __init__(self, name: str, contents: Dict[str, int]):
        self.name = name
        self.contents = contents

    @staticmethod
    def from_spec(spec: str) -> "Bag":
        spec_match = match(r"^([\w\s]+) bags contain (.*).$", spec)
        name = spec_match[1]
        contents = {
            item[1]: int(item[0])
            for item in findall(r"(\d+) ([\w\s]+) bags?", spec_match[2])
        }

        return Bag(name, contents)
    

test_bag = Bag.from_spec("light red bags contain 1 bright white bag, 2 muted yellow bags.")
assert test_bag.name == "light red"
assert test_bag.contents == { "bright white": 1, "muted yellow": 2 }

example_bags = [Bag.from_spec(spec) for spec in example]


In [31]:
from typing import List
from collections import defaultdict

class BagMap(object):
    def __init__(self, bags: List[Bag]):
        self.bags = bags
        self.bag_lookup: Dict[str, Dict[str, int]] = defaultdict(lambda: {})

        for bag in bags:
            for child_name, count in bag.contents.items():
                self.bag_lookup[bag.name][child_name] = count

                self._update_parents(bag.name, child_name, count)

        for bag_type, bag_contents in self.bag_lookup.items():
            for child_type, count in bag_contents.items():
                self._update_parents(bag_type, child_type, count)

    def get_containers(self, name: str) -> List[str]:
        for bag_name, contents in self.bag_lookup.items():
            if name in contents and contents[name] > 0:
                yield bag_name

    def get_child_count(self, name: str) -> int:
        total_bags = 0
        bag = next(bag for bag in self.bags if bag.name == name)
        for bag_type, count in bag.contents.items():
            extra_bags = count * (1 + self.get_child_count(bag_type))
            total_bags += extra_bags

        return total_bags

    def _update_parents(self, bag_type: str, child_type: str, child_count: int):
        for parent_type, parent_contents in self.bag_lookup.items():
            if parent_type == bag_type:
                continue

            if bag_type in parent_contents:
                old_value = parent_contents.get(child_type, 0)
                new_value = max(parent_contents[bag_type] * child_count, old_value)

                if old_value != new_value:
                    parent_contents[child_type] = new_value
                    self._update_parents(parent_type, child_type, new_value)

example_map = BagMap(example_bags)
assert set(example_map.get_containers("shiny gold")) == set(("bright white", "muted yellow", "dark orange", "light red"))

print(set(example_map.get_containers("shiny gold")))

{'light red', 'muted yellow', 'dark orange', 'bright white'}


In [33]:
true_bags = [Bag.from_spec(spec) for spec in data]
print(f"Total Bags: {len(true_bags)}")
true_map = BagMap(true_bags)
print(f"Total Containers: {len(set(true_map.get_containers('shiny gold')))}")

Total Bags: 594
Total Containers: 115


In [34]:
print(f"One shiny gold bag must contain {example_map.get_child_count('shiny gold')} other bags")
assert example_map.get_child_count("shiny gold") == 32

example2 = """
shiny gold bags contain 2 dark red bags.
dark red bags contain 2 dark orange bags.
dark orange bags contain 2 dark yellow bags.
dark yellow bags contain 2 dark green bags.
dark green bags contain 2 dark blue bags.
dark blue bags contain 2 dark violet bags.
dark violet bags contain no other bags.
""".strip().splitlines()


example_bags2 = [Bag.from_spec(spec) for spec in example2]
example_map2 = BagMap(example_bags2)
print(f"One shiny gold bag must contain {example_map2.get_child_count('shiny gold')} other bags")
assert example_map2.get_child_count("shiny gold") == 126

One shiny gold bag must contain 32 other bags
One shiny gold bag must contain 126 other bags


In [35]:
print(f"One shiny gold bag must contain {true_map.get_child_count('shiny gold')} other bags")

One shiny gold bag must contain 1250 other bags
